In [1]:
import sys, os
import torch
import time
import random
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from collections import defaultdict, Counter
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
cuda = torch.device('cuda:1')

In [3]:
class AE(nn.Module):
    """
    AutoEncoder
    """

    def __init__(self, d):
        super(AE, self).__init__()
        self.d = d
        nodes = 2
        if d > 100:
            nodes = 8
        self.ae = nn.Sequential(
            nn.Linear(d, nodes),
            nn.Linear(nodes, nodes),
            nn.Linear(nodes, d)
        ).to(cuda)

        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight).to(cuda)

    def forward(self, x):
        x = x.view(-1, self.d)
        x1 = self.ae(x)
        return x1.view(-1, 1, self.d)

In [4]:
def Train(model, dset, alpha, gamma, train_input, attribute, majority, minority, epochs, batch, 
          base_score, one_hot, IDCG, PV_assignment, fair = False):
    """
    Model Training
    Args:
        model: model to be trained
        dset: dataset
        alpha: hyperparameter1
        gamma: hyperparameter2
        train_input: sample
        attribute: sensitive attribute subgroups
        majority: the majority subgroup
        minority: the minority subgroup
        epochs: training epoches
        batch: minibatch size
        base_score: the preobtained outlier score from the AE
        one_hot: one-hot-encoding of sensitive attribute subgroups
        IDCG: IDCG for each protected subgroup
        PV_assignment: The dict indicating each subgroup's index information, i.e., 1th, 3th, 5th belonging to male, 2th, 4th, 6th belonging to female
        fair: if False, will only train a standard AE, else will train FairOD.
                Note that FairOD require the output of AE
    Returns: statistical parity, group fidelity, and outlier score
    """
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    running_loss = 0
    for epoch in range(epochs):
        for e in range(train_input.shape[0] // batch):
            cur_index = list(range(e * batch, (e + 1) * batch))
            input_batch = train_input[cur_index]
            x = torch.tensor(input_batch).float()
            x = x.to(cuda)

            x1 = model(x)
            if x.shape != x1.shape:
                x = np.reshape(x.data.cpu().numpy(), x1.shape)
                x = Variable(torch.tensor(x)).to(cuda)
            self_reconstruction_loss = nn.functional.mse_loss(x1, x, reduction='none').to(cuda)
            self_reconstruction_loss = torch.sum(self_reconstruction_loss, dim=2).to(cuda)
            self_reconstruction_loss = torch.reshape(self_reconstruction_loss, (self_reconstruction_loss.shape[0],))

            if fair:
                """
                in the paper: 
                one could one-hot-encode (OHE) the 𝑃𝑉 into multiple variables and minimize the correlation of 
                outlier scores with each variable additively. That is, an outer sum would be added to 
                Eq. (12) that goes over the new OHE variables encoding the categorical 𝑃𝑉 .
                """
                os_mean = torch.mean(self_reconstruction_loss)
                os_std = torch.std(self_reconstruction_loss)
                one_hot = torch.tensor(one_hot).to(cuda)
                Statistical_Parity = 0

                for i in range(one_hot.shape[1]):
                    cur_group = one_hot[:, i]
                    pv_mean = torch.mean(cur_group)
                    pv_std = torch.std(cur_group)
                    Statistical_Parity += abs(
                        (torch.sum(self_reconstruction_loss) - os_mean) * (torch.sum(cur_group) - pv_mean) / (os_std * pv_std))

                Group_Fidelity = len(set(attribute))
                for i in set(attribute):
                    group_batch_index = list(set().intersection(PV_assignment[i], cur_index))
                    for k in group_batch_index:
                        j = k - e * batch
                        Group_Fidelity -= (np.power(2, base_score[k]) - 1) / (IDCG[i] * torch.log2(1 +
                                                                torch.sum(torch.sigmoid(self_reconstruction_loss[group_batch_index] - self_reconstruction_loss[j]))))

                L = alpha * self_reconstruction_loss.mean() + (1 - alpha) * Statistical_Parity + gamma * Group_Fidelity
            else:
                L = self_reconstruction_loss.mean()
            optimizer.zero_grad()
            L.backward()
            optimizer.step()
            running_loss += L.data.cpu().numpy()
#             if e % 10 == 9:  # show loss every 30 mini-batches
#                 print(f'[{epoch + 1},     {e + 1}] loss:{running_loss / 10:.2f}')
#                 running_loss = 0.0
    
    
    total = torch.tensor(train_input).float()
    total = total.to(cuda)
    final_x1 = model(total)
    if final_x1.shape != total.shape:
        total = np.reshape(total.data.cpu().numpy(), final_x1.shape)
        total = Variable(torch.tensor(total)).to(cuda)
    final_score = nn.functional.mse_loss(final_x1, total, reduction='none').to(cuda)
    final_score = torch.sum(final_score, dim=2).to(cuda)
    final_score = torch.reshape(final_score, (final_score.shape[0],))
    if not fair:
        np.save(f'data/{dset}.npy', final_score.data.cpu().numpy())
        return final_score.data.cpu().numpy()
    else:
        percentage = 95
        final_score_numpy = final_score.data.cpu().numpy()
        threshold = np.percentile(final_score_numpy, q=percentage)
        a = 0
        b = 0
        for i in range(train_input.shape[0]):
            if final_score_numpy[i] > threshold:
                if attribute[i] == majority[0]:
                    a += 1
                elif attribute[i] == minority[0]:
                    b += 1
        r = (a / majority[1]) / (b / minority[1])
        fairness = min(r, 1/r)

        NDCG_a = 0
        NDCG_b = 0
        for j in PV_assignment[majority[0]]:
            sum = 1 + torch.sum(final_score[PV_assignment[majority[0]]] >= final_score[j])
            NDCG_a += (torch.pow(2, torch.tensor(base_score[j]).to(cuda)) - 1) / (IDCG[majority[0]] * torch.log2(sum.float()))

        for j in PV_assignment[minority[0]]:
            sum = 1 + torch.sum(final_score[PV_assignment[minority[0]]] >= final_score[j])
            NDCG_b += (torch.pow(2, torch.tensor(base_score[j]).to(cuda)) - 1) / (IDCG[minority[0]] * torch.log2(sum.float()))
        group_fidelity = 1 / (1 / NDCG_a.data.cpu().numpy() + 1/NDCG_b.data.cpu().numpy())
        print('fairness:', fairness)
        print('group_fidelity:', group_fidelity)
        return fairness, group_fidelity, final_score.data.cpu().numpy()
    
def evalation(model, df_test, df_test_cf):
    model.eval()
    test_X = torch.tensor(df_test.iloc[:,1:-1].values.astype(np.float32)).float()
    test_X = test_X.to(cuda)
    test_X_hat = model(test_X)
    if test_X_hat.shape != test_X.shape:
        test_X = np.reshape(test_X.data.cpu().numpy(), test_X_hat.shape)
        test_X = Variable(torch.tensor(test_X)).to(cuda)
    final_score = nn.functional.mse_loss(test_X_hat, test_X, reduction='none').to(cuda)
    final_score = torch.sum(final_score, dim=2).to(cuda)
    final_score = torch.reshape(final_score, (final_score.shape[0],))
    
    test_X_cf = torch.tensor(df_test_cf.values.astype(np.float32)).float()
    test_X_cf = test_X_cf.to(cuda)
    test_X_cf_hat = model(test_X_cf)
    if test_X_cf_hat.shape != test_X_cf.shape:
        test_X_cf = np.reshape(test_X_cf.data.cpu().numpy(), test_X_cf_hat.shape)
        test_X_cf = Variable(torch.tensor(test_X_cf)).to(cuda)
    final_score_cf = nn.functional.mse_loss(test_X_cf_hat, test_X_cf, reduction='none').to(cuda)
    final_score_cf = torch.sum(final_score_cf, dim=2).to(cuda)
    final_score_cf = torch.reshape(final_score_cf, (final_score_cf.shape[0],))
    return final_score.data.cpu().numpy(), final_score_cf.data.cpu().numpy()
    
def set_seed(seed = 0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
def load_data(adult=0):
    if not adult:
        df_train = pd.read_csv('data/train.csv', index_col=0)
        df_test = pd.read_csv('data/test.csv', index_col=0)
        df_test_cf = pd.read_csv('data/test_cf.csv', index_col=0)
    else:
        df_train = pd.read_csv('data/adult_train.csv', index_col=0)
        df_test = pd.read_csv('data/adult_test.csv', index_col=0)
        df_test_cf = pd.read_csv('data/adult_do.csv', index_col=0)
    return df_train, df_test, df_test_cf

def get_thres(score, quantile=0.95):
    return np.quantile(score, quantile)

def get_fairness_res(test_Y, score, score_cf, thres):
    print("F")
    pr = average_precision_score(y_true=test_Y, y_score=score)
    roc = roc_auc_score(test_Y, score)
    print('PR:', pr)
    print('AUC-ROC:', roc)
    pred = [1 if i > thres else 0 for i in score]
    df_org = pd.DataFrame()
    df_org['label'] = test_Y
    df_org['pred'] = pred
    print(classification_report(test_Y, pred, digits=5))
    print(confusion_matrix(test_Y, pred))

    
    print('-'*60)
    print('CF')
    print('PR:', average_precision_score(y_true=test_Y, y_score=score_cf))
    print('AUC-ROC:', roc_auc_score(test_Y, score_cf))
    pred = [1 if i > thres else 0 for i in score_cf]
    df_org['pred_cf'] = pred
    print(classification_report(test_Y, pred, digits=5))
    print(confusion_matrix(test_Y, pred))
    total = len(df_org)
    df_org['cf_changed'] = df_org['pred_cf'] - df_org['pred']
    df_org_cf = df_org.groupby(['cf_changed']).count().reset_index(drop=False)
    before_cf = sum(df_org_cf.loc[df_org_cf['cf_changed'] != 0]['label'].values)
    cr = before_cf / total
    print(f'The prediction changed: {cr}')
    return df_org, pr, roc, cr

In [14]:
set_seed(0)
quantile = 0.999
db = 'adult'
df_train, df_test, df_test_cf = load_data(1)
X_norm = df_train.iloc[:,1:-1].values.astype(np.float32)
Y = df_train['y'].values
sensitive_attribute_group = df_train.iloc[:,0].values
input = np.reshape(sensitive_attribute_group, (-1, 1))
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(input)
one_hot = enc.transform(input).toarray()
sensitive_attribute_group = np.argmax(one_hot, axis=1)

# -----data shuffling----- #
set_seed(0)
random_index = np.random.permutation(X_norm.shape[0])
X_norm = X_norm[random_index]
Y = Y[random_index]
sensitive_attribute_group = sensitive_attribute_group[random_index]

# -----setting epoch and minibatch size----- #
starttime = time.time()
configuration = 90, 64

fair_command = 'f'

test_Y = df_test['y']

# -----train AE----- #
if fair_command == 'f':
    ae = AE(X_norm.shape[1]).to(cuda)
    score = Train(ae, db, 0, 0, X_norm, [], [], [], configuration[0], configuration[1], NotImplemented, NotImplemented, NotImplemented, NotImplemented, fair=False)
    thres = get_thres(score, quantile)
    score, score_cf = evalation(ae, df_test, df_test_cf)
    df_org,_,_,_ = get_fairness_res(test_Y, score, score_cf, thres)
          

else:
    fair = True

    
    
# -----cache IDCG and PV_assignment to facilitate model training----- #
fair = True
base_score = np.load(f'data/{db}.npy')
base_score = (base_score - min(base_score)) / (max(base_score) - min(base_score))
IDCG = defaultdict(float)
PV_assignment = defaultdict(list)
for i in range(X_norm.shape[0]):
    PV_assignment[sensitive_attribute_group[i]].append(i)

for i in set(sensitive_attribute_group):
    scores = torch.tensor(base_score[PV_assignment[i]]).to(cuda)
    scores, _ = torch.sort(scores, descending=True)
    scores = scores.data.cpu().numpy()
    for j in range(len(scores)):
        IDCG[i] += (np.power(2, scores[j]) - 1) / np.log2(1 + (j + 1))

c = Counter(sensitive_attribute_group)
majority = c.most_common()[0]
minority = c.most_common()[-1]

# alpha = [0.01, 0.9]
alpha = [0.9]
# gamma = [0.01, 1.0]
gamma = [0.01]
best_choice = NotImplemented
lst_pr = []
lst_roc = []
lst_cr = []

for i in alpha:
    for j in gamma:
        print(f'Using alpha: {i},   gamma:{j}')
        ae = AE(X_norm.shape[1]).to(cuda)
        if db == 'kdd':
            ae = nn.DataParallel(ae, device_ids=devices).to(cuda)
        fairness, group_fidelity, final_scores = \
            Train(ae, db, i, j, X_norm, sensitive_attribute_group, majority, minority, configuration[0], configuration[1], base_score, one_hot, IDCG, PV_assignment, fair=fair)
        # -----find the best combination of alpha and gamma in the hyperparameter grid ----- #
        if best_choice is NotImplemented:
            best_choice = (fairness, group_fidelity, final_scores, ae)
        else:
            current_best = best_choice[0] + best_choice[1]
            if current_best < fairness + group_fidelity:
                best_choice = (fairness, group_fidelity, final_scores, ae)
        thres = get_thres(best_choice[2], quantile)
        score, score_cf = evalation(best_choice[3], df_test, df_test_cf)
        df_fair, pr, roc, cr = get_fairness_res(test_Y, score, score_cf, thres)
        lst_pr.append(pr)
        lst_roc.append(roc)
        lst_cr.append(cr)
print('Final PR:', np.array(lst_pr).mean())
print('Final ROC:', np.array(lst_roc).mean())
print('FInal CR:', np.array(lst_cr).mean())

F
PR: 0.2642230167279809
AUC-ROC: 0.6564716319444445
              precision    recall  f1-score   support

           0    0.83684   0.99883   0.91069     12000
           1    0.81818   0.02625   0.05087      2400

    accuracy                        0.83674     14400
   macro avg    0.82751   0.51254   0.48078     14400
weighted avg    0.83373   0.83674   0.76738     14400

[[11986    14]
 [ 2337    63]]
------------------------------------------------------------
CF
PR: 0.25471795920629503
AUC-ROC: 0.6408912326388889
              precision    recall  f1-score   support

           0    0.83730   0.99492   0.90933     12000
           1    0.56738   0.03333   0.06297      2400

    accuracy                        0.83465     14400
   macro avg    0.70234   0.51413   0.48615     14400
weighted avg    0.79231   0.83465   0.76827     14400

[[11939    61]
 [ 2320    80]]
The prediction changed: 0.005555555555555556
Using alpha: 0.9,   gamma:0.01


/tmp/ipykernel_2896821/3874804019.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  one_hot = torch.tensor(one_hot).to(cuda)


fairness: 0.8477422533297092
group_fidelity: 0.4595331000986003
F
PR: 0.18846979288890897
AUC-ROC: 0.5637683506944445
              precision    recall  f1-score   support

           0    0.83332   0.99783   0.90819     12000
           1    0.16129   0.00208   0.00411      2400

    accuracy                        0.83188     14400
   macro avg    0.49731   0.49996   0.45615     14400
weighted avg    0.72132   0.83188   0.75751     14400

[[11974    26]
 [ 2395     5]]
------------------------------------------------------------
CF
PR: 0.18957472531099023
AUC-ROC: 0.5442273958333333
              precision    recall  f1-score   support

           0    0.83349   0.98817   0.90426     12000
           1    0.17919   0.01292   0.02410      2400

    accuracy                        0.82563     14400
   macro avg    0.50634   0.50054   0.46418     14400
weighted avg    0.72444   0.82563   0.75757     14400

[[11858   142]
 [ 2369    31]]
The prediction changed: 0.01138888888888889
Final 